In [ ]:
import geopandas
import numpy
import pandas
import rasterio
from pyproj import Geod
from tqdm import tqdm as tqdm_core
from tqdm.notebook import tqdm

In [ ]:
import math
import os
from glob import glob

In [ ]:
tqdm.pandas()

In [ ]:
# Read data
sorted(glob("outputs/*"))

In [ ]:
continent = 'asia'

In [ ]:
df = geopandas.read_parquet(f'outputs/{continent}-latest-highway-core_splits.geoparquet')

In [ ]:
df.columns

In [ ]:
# Calculate min/max over exposure
exposure = df[df.columns[3:]]
df['min_exp'] = exposure.min(axis=1)
df['max_exp'] = exposure.max(axis=1)

In [ ]:
# Calculate line length on the ellipsoid
geod = Geod(ellps="WGS84")
df['length'] = df.geometry.progress_apply(geod.geometry_length)

In [ ]:
# Extract only exposed
any_exposed = df[df.max_exp > 0].copy()

In [ ]:
len(any_exposed), len(df)

In [ ]:
sorted(any_exposed.columns)

In [ ]:
any_exposed.head()

# Stats

In [ ]:
hist = df[[
    'inunriver_historical_000000000WATCH_1980_rp00050',
    'inunriver_historical_000000000WATCH_1980_rp00100',
    'inunriver_historical_000000000WATCH_1980_rp00500',
    'inunriver_historical_000000000WATCH_1980_rp01000',
    'length'
]]

In [ ]:
cols = [
    (50, 'inunriver_historical_000000000WATCH_1980_rp00050'),
    (100, 'inunriver_historical_000000000WATCH_1980_rp00100'),
    (500, 'inunriver_historical_000000000WATCH_1980_rp00500'),
    (1000, 'inunriver_historical_000000000WATCH_1980_rp01000'),
]

In [ ]:
for _, col in cols:
    print(len(hist[hist[col].isna()]))

In [ ]:
total = df['length'].sum() / 1e3
print(f"Total roads in {continent} {total:0,.0f}km")

In [ ]:
bands = [
    (float('-inf'), 0),
    (0, 1),
    (1, 2),
    (2, float('inf'))
]

In [ ]:
summary = []
for lower, upper in bands:
    for rp, col in cols:
        l = hist[(hist[col] > lower) & (hist[col] <= upper)]['length'].sum() / 1e3
        summary.append({
            "depth_band": f"{lower}-{upper}m",
            "road_length_km": l,
            "return_period": rp
        })
summary = pandas.DataFrame(summary).pivot(columns="depth_band", index="return_period")
summary.to_csv(f"summary_historical_{continent}.csv")
summary

In [ ]:
rcp_cols = sorted([c for c in df.columns if "inunriver_rcp" in c])

summary = []
for lower, upper in bands:
    for col in rcp_cols:
        _, rcp, model, _, rp = col.split("_")
        rp = int(rp[2:])
        model = model.replace("0","")
        
        l = df[(df[col] > lower) & (df[col] <= upper)]['length'].sum() / 1e3
        summary.append({
            "depth_band": f"{lower}-{upper}m",
            "model": model,
            "rcp": rcp,
            "return_period": rp,
            "road_length_km": l
        })
summary = pandas.DataFrame(summary).pivot(columns=["depth_band"], index=["rcp", "return_period", "model"])
summary.to_csv(f"summary_future_{continent}.csv")
summary

In [ ]:
# Convert index to tuple so we can hash it and do a groupby
any_exposed.cell_index = any_exposed.cell_index.apply(tuple)

In [ ]:
# Calculate exposed length per cell
length_per_cell = any_exposed[['cell_index', 'length']].groupby('cell_index').sum()

In [ ]:
# Back to raster
with rasterio.open('../aqueduct/inuncoast_historical_nosub_hist_rp0001_5.tif') as dataset:
    raster_width = dataset.width
    raster_height = dataset.height
    raster_transform = dataset.transform

In [ ]:
length_raster = numpy.zeros((raster_height, raster_width))

In [ ]:
for cell in length_per_cell.reset_index().itertuples():
    col, row = cell.cell_index
    length_raster[row, col] = cell.length

In [ ]:
with rasterio.open(
        f'outputs/{continent}-core.tif',
        'w',
        driver='GTiff',
        height=length_raster.shape[0],
        width=length_raster.shape[1],
        count=1,
        dtype=length_raster.dtype,
        crs='+proj=latlong',
        transform=raster_transform,
        compress='lzw'
    ) as dataset:
    dataset.write(length_raster, 1)

In [ ]:
# Downsample

In [ ]:
def downsample(df, factor, raster_height, raster_width, raster_transform):
    # Set up rescaled transform
    height_ds = math.floor(raster_height * factor)
    width_ds = math.floor(raster_width * factor)
    raster_transform_ds = raster_transform * raster_transform.scale(
        (raster_width / width_ds),
        (raster_height / height_ds)
    ) 
    
    # Downsample
    def downsample_index(xy):
        x, y = xy
        x = math.floor(x * factor) % width_ds
        y = math.floor(y * factor) % height_ds
        return (x, y)
    df['cell_index_downsample'] = df.cell_index.apply(downsample_index)
    grouped = df[['cell_index_downsample', 'length']].groupby('cell_index_downsample').sum()
    
    # Set up data array
    length_raster_ds = numpy.zeros((height_ds, width_ds))    
    for cell in grouped.reset_index().itertuples():
        col, row = cell.cell_index_downsample
        length_raster_ds[row, col] = cell.length
        
    with rasterio.open(
        f'outputs/{continent}-core_ds_{factor}.tif',
        'w',
        driver='GTiff',
        height=length_raster_ds.shape[0],
        width=length_raster_ds.shape[1],
        count=1,
        dtype=length_raster_ds.dtype,
        crs='+proj=latlong',
        transform=raster_transform_ds,
        compress='lzw'
    ) as dataset:
        dataset.write(length_raster_ds, 1)

In [ ]:
downsample(any_exposed, 1/16, raster_height, raster_width, raster_transform)

In [ ]:
downsample(any_exposed, 1/32, raster_height, raster_width, raster_transform)

In [ ]:
# Finally output all exposed vector

In [ ]:
any_exposed.columns

In [ ]:
any_exposed.drop(columns=['cell_index', 'cell_index_downsample']).to_file(f'/tmp/mert2014/outputs/{continent}-latest-highway-core_splits_exposed.gpkg', driver="GPKG")